In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path 
#from rich import inspect

In [34]:
## Build'n'Load Stuff (paths and arrays)
freemocap_validation_data_path = Path(r"C:\Users\aaron\Documents\HumonLab\Spring2022\ValidationStudy\FreeMocap_Data")
sessionID = 'session_SER_1_20_22'
this_freemocap_session_path = freemocap_validation_data_path / sessionID
this_freemocap_data_path = this_freemocap_session_path/'DataArrays'
mediapipe_data_path = this_freemocap_data_path / 'mediaPipeSkel_3d_smoothed.npy'

In [35]:

mediapipeSkel_fr_mar_dim = np.load(mediapipe_data_path)

In [36]:
num_pose_joints = 33
first_frame = 0
last_frame = 100

pose_joint_range = range(num_pose_joints)
frame_range = range(first_frame,last_frame)

mediapipe_pose_data = mediapipeSkel_fr_mar_dim[frame_range[0]:frame_range[-1],pose_joint_range[0]:pose_joint_range[-1],:]
skel_x = mediapipe_pose_data[:,:,0]
skel_y = mediapipe_pose_data[:,:,1]
skel_z = mediapipe_pose_data[:,:,2]

In [18]:
# good_frame = 10439
# mediapipe_good_frame_data = mediaPipeSkel_fr_mar_dim[good_frame,:,:]
# skel_x = mediapipe_good_frame_data[:,0]
# skel_y = mediapipe_good_frame_data[:,1]
# skel_z = mediapipe_good_frame_data[:,2]

In [37]:
%matplotlib
plt.ion()

Using matplotlib backend: <object object at 0x00000227700F7170>


In [38]:
figure = plt.figure()
ax = figure.add_subplot(111, projection='3d')
# inspect(ax,methods= True)

In [39]:
ax.scatter(skel_x, skel_y, skel_z)
plt.show()